# Regiment

### Introduction:

Special thanks to: http://chrisalbon.com/ for sharing the dataset and materials.

### Step 1. Import the necessary libraries

In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, min, avg,max

### Step 2. Create the DataFrame with the following values:

In [2]:
raw_data = {'regiment': ['Nighthawks', 'Nighthawks', 'Nighthawks', 'Nighthawks', 'Dragoons', 'Dragoons', 'Dragoons', 'Dragoons', 'Scouts', 'Scouts', 'Scouts', 'Scouts'], 
        'company': ['1st', '1st', '2nd', '2nd', '1st', '1st', '2nd', '2nd','1st', '1st', '2nd', '2nd'], 
        'name': ['Miller', 'Jacobson', 'Ali', 'Milner', 'Cooze', 'Jacon', 'Ryaner', 'Sone', 'Sloan', 'Piger', 'Riani', 'Ali'], 
        'preTestScore': [4, 24, 31, 2, 3, 4, 24, 31, 2, 3, 2, 3],
        'postTestScore': [25, 94, 57, 62, 70, 25, 94, 57, 62, 70, 62, 70]}

In [3]:
spark = SparkSession.builder\
                    .appName('regiment')\
                    .getOrCreate()

25/06/11 14:28:58 WARN Utils: Your hostname, kevin-llanos-Type1ProductConfigId resolves to a loopback address: 127.0.1.1; using 192.168.1.92 instead (on interface wlo1)
25/06/11 14:28:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/11 14:28:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Step 3. Assign it to a variable called regiment.
#### Don't forget to name each column

In [4]:
df = pd.DataFrame(raw_data)

In [5]:
regiment = spark.createDataFrame(df)


In [6]:
regiment.show()

+----------+-------+--------+------------+-------------+
|  regiment|company|    name|preTestScore|postTestScore|
+----------+-------+--------+------------+-------------+
|Nighthawks|    1st|  Miller|           4|           25|
|Nighthawks|    1st|Jacobson|          24|           94|
|Nighthawks|    2nd|     Ali|          31|           57|
|Nighthawks|    2nd|  Milner|           2|           62|
|  Dragoons|    1st|   Cooze|           3|           70|
|  Dragoons|    1st|   Jacon|           4|           25|
|  Dragoons|    2nd|  Ryaner|          24|           94|
|  Dragoons|    2nd|    Sone|          31|           57|
|    Scouts|    1st|   Sloan|           2|           62|
|    Scouts|    1st|   Piger|           3|           70|
|    Scouts|    2nd|   Riani|           2|           62|
|    Scouts|    2nd|     Ali|           3|           70|
+----------+-------+--------+------------+-------------+



### Step 4. What is the mean preTestScore from the regiment Nighthawks?  

In [7]:
regiment.filter(col('regiment')=='Nighthawks').agg(avg(col('postTestScore')).alias('Promedio')).show()

+--------+
|Promedio|
+--------+
|    59.5|
+--------+



In [8]:
df_night = df.loc[df['regiment']=='Nighthawks', :]

promedio_night = df_night['postTestScore'].mean()

print('El promedio de prestescore para nightwaks %s' %promedio_night)

El promedio de prestescore para nightwaks 59.5


### Step 5. Present general statistics by company

In [9]:
from pyspark.sql.functions import avg, stddev, min, max, count

grouped_company = regiment.groupBy(col('company'))

estadisticas = grouped_company.agg(
    count("*").alias("n"),
    avg("preTestScore").alias("_prom"),
    stddev("preTestScore").alias("_std"),
    min("preTestScore").alias("_min"),
    max("preTestScore").alias("_max"),
    avg("postTestScore").alias("_prom"),
    stddev("postTestScore").alias("_std"),
     min("postTestScore").alias("_min"),
    max("postTestScore").alias("_max"),
    avg("postTestScore").alias("_prom"),
)

estadisticas.show()

+-------+---+-----------------+------------------+----+----+------------------+------------------+----+----+------------------+
|company|  n|            _prom|              _std|_min|_max|             _prom|              _std|_min|_max|             _prom|
+-------+---+-----------------+------------------+----+----+------------------+------------------+----+----+------------------+
|    1st|  6|6.666666666666667| 8.524474568362947|   2|  24|57.666666666666664| 27.48575388572536|  25|  94|57.666666666666664|
|    2nd|  6|             15.5|14.652644812456213|   2|  31|              67.0|14.057026712644463|  57|  94|              67.0|
+-------+---+-----------------+------------------+----+----+------------------+------------------+----+----+------------------+



In [10]:
df.groupby('company').describe()

preTestScore                                                      \
               count       mean        std  min   25%   50%    75%   max   
company                                                                    
1st              6.0   6.666667   8.524475  2.0  3.00   3.5   4.00  24.0   
2nd              6.0  15.500000  14.652645  2.0  2.25  13.5  29.25  31.0   

        postTestScore                                                       
                count       mean        std   min    25%   50%   75%   max  
company                                                                     
1st               6.0  57.666667  27.485754  25.0  34.25  66.0  70.0  94.0  
2nd               6.0  67.000000  14.057027  57.0  58.25  62.0  68.0  94.0

### Step 6. What is the mean of each company's preTestScore?

In [11]:
grouped_company = regiment.groupBy(col('company'))

estadisticas = grouped_company.agg(
    avg("preTestScore").alias("pretestscore_prom"),
)

estadisticas.show()

+-------+-----------------+
|company|pretestscore_prom|
+-------+-----------------+
|    1st|6.666666666666667|
|    2nd|             15.5|
+-------+-----------------+



In [12]:
df.groupby('company')['preTestScore'].mean()

company
1st     6.666667
2nd    15.500000
Name: preTestScore, dtype: float64

25/06/11 14:29:13 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


### Step 7. Present the mean preTestScores grouped by regiment and company

In [ ]:
from pyspark.sql import functions as F



regiment.groupBy(F.col('regiment'), F.col('company')).agg(F.avg(F.col('preTestScore')).alias('Promedio_pretestscore')).show()




+----------+-------+---------------------+
|  regiment|company|Promedio_pretestscore|
+----------+-------+---------------------+
|Nighthawks|    1st|                 14.0|
|Nighthawks|    2nd|                 16.5|
|  Dragoons|    1st|                  3.5|
|  Dragoons|    2nd|                 27.5|
|    Scouts|    1st|                  2.5|
|    Scouts|    2nd|                  2.5|
+----------+-------+---------------------+



In [17]:
df.groupby(
    ['regiment','company'], as_index=False
).agg(promedio=('preTestScore', 'mean'))




,regiment,company,promedio
0,Dragoons,1st,3.5
1,Dragoons,2nd,27.5
2,Nighthawks,1st,14.0
3,Nighthawks,2nd,16.5
4,Scouts,1st,2.5
5,Scouts,2nd,2.5


### Step 8. Present the mean preTestScores grouped by regiment and company without heirarchical indexing

In [19]:
pivoted_df = regiment.groupBy("regiment") \
    .pivot("company") \
    .agg(F.avg("preTestScore").alias("Promedio_pretestscore"))
pivoted_df.show()


+----------+----+----+
|  regiment| 1st| 2nd|
+----------+----+----+
|Nighthawks|14.0|16.5|
|  Dragoons| 3.5|27.5|
|    Scouts| 2.5| 2.5|
+----------+----+----+



In [20]:
df.groupby(
    ['regiment','company'], as_index=False
).agg(promedio=('preTestScore', 'mean')).pivot(index=['regiment'], columns='company', values='promedio')



company,1st,2nd
regiment,,
Dragoons,3.5,27.5
Nighthawks,14.0,16.5
Scouts,2.5,2.5


### Step 9. Group the entire dataframe by regiment and company

In [23]:
grouped_data = regiment.groupBy(F.col('regiment'), F.col('company'))\
                        .agg(F.avg('preTestScore').alias('promedio_pre'), 
                             F.avg('postTestScore').alias('promedio_post'))


grouped_data.show()



+----------+-------+------------+-------------+
|  regiment|company|promedio_pre|promedio_post|
+----------+-------+------------+-------------+
|Nighthawks|    1st|        14.0|         59.5|
|Nighthawks|    2nd|        16.5|         59.5|
|  Dragoons|    1st|         3.5|         47.5|
|  Dragoons|    2nd|        27.5|         75.5|
|    Scouts|    1st|         2.5|         66.0|
|    Scouts|    2nd|         2.5|         66.0|
+----------+-------+------------+-------------+



In [28]:

df['postTestScore'] = pd.to_numeric(df['postTestScore'], errors='coerce')
df['preTestScore'] = pd.to_numeric(df['preTestScore'], errors='coerce')

df.groupby(['regiment', 'company']).mean(numeric_only=True)




preTestScore  postTestScore
regiment   company                             
Dragoons   1st               3.5           47.5
           2nd              27.5           75.5
Nighthawks 1st              14.0           59.5
           2nd              16.5           59.5
Scouts     1st               2.5           66.0
           2nd               2.5           66.0

### Step 10. What is the number of observations in each regiment and company

In [37]:
regiment.groupBy(F.col('company'), F.col('regiment')).count().show()


+-------+----------+-----+
|company|  regiment|count|
+-------+----------+-----+
|    1st|Nighthawks|    2|
|    2nd|Nighthawks|    2|
|    1st|  Dragoons|    2|
|    2nd|  Dragoons|    2|
|    1st|    Scouts|    2|
|    2nd|    Scouts|    2|
+-------+----------+-----+



In [38]:
df.groupby(['company', 'regiment']).size()

company  regiment  
1st      Dragoons      2
         Nighthawks    2
         Scouts        2
2nd      Dragoons      2
         Nighthawks    2
         Scouts        2
dtype: int64

### Step 11. Iterate over a group and print the name and the whole data from the regiment

In [47]:

regiment_lista = [row['regiment']  for row in regiment.select('regiment').distinct().collect()]



for regimiento in regiment_lista:
    
    print(f'Para el regimiento {regimiento} \n')
    
    regiment.filter(F.col('regiment') == regimiento).show()

Para el regimiento Nighthawks 

+----------+-------+--------+------------+-------------+
|  regiment|company|    name|preTestScore|postTestScore|
+----------+-------+--------+------------+-------------+
|Nighthawks|    1st|  Miller|           4|           25|
|Nighthawks|    1st|Jacobson|          24|           94|
|Nighthawks|    2nd|     Ali|          31|           57|
|Nighthawks|    2nd|  Milner|           2|           62|
+----------+-------+--------+------------+-------------+

Para el regimiento Dragoons 

+--------+-------+------+------------+-------------+
|regiment|company|  name|preTestScore|postTestScore|
+--------+-------+------+------------+-------------+
|Dragoons|    1st| Cooze|           3|           70|
|Dragoons|    1st| Jacon|           4|           25|
|Dragoons|    2nd|Ryaner|          24|           94|
|Dragoons|    2nd|  Sone|          31|           57|
+--------+-------+------+------------+-------------+

Para el regimiento Scouts 

+--------+-------+-----+----

In [53]:
regimiento_list = df['regiment'].unique().tolist()



for regimiento in regimiento_list:
    
    print(f'Para el regimiento {regimiento} \n')
    
    print(df.loc[df['regiment']==regimiento, :])


Para el regimiento Nighthawks 

     regiment company      name  preTestScore  postTestScore
0  Nighthawks     1st    Miller             4             25
1  Nighthawks     1st  Jacobson            24             94
2  Nighthawks     2nd       Ali            31             57
3  Nighthawks     2nd    Milner             2             62
Para el regimiento Dragoons 

   regiment company    name  preTestScore  postTestScore
4  Dragoons     1st   Cooze             3             70
5  Dragoons     1st   Jacon             4             25
6  Dragoons     2nd  Ryaner            24             94
7  Dragoons     2nd    Sone            31             57
Para el regimiento Scouts 

   regiment company   name  preTestScore  postTestScore
8    Scouts     1st  Sloan             2             62
9    Scouts     1st  Piger             3             70
10   Scouts     2nd  Riani             2             62
11   Scouts     2nd    Ali             3             70
